# Create a Local Ethernet (Layer 2) Network

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it.  


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                         
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
Bastion Username,durbek_gafurov_0000000854
Bastion Private Key File,/home/fabric/work/fabric_config/bastionD
Bastion Host,bastion-1.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/.ssh/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/.ssh/slice_key
Slice Private Key Passphrase,None


## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
STAR,"710 North Lake Shore Dr,Chicago, IL 60611","(41.89537135, -87.61663220067463)",6,12,384,384,0,3072,3072,0,121200,121200,0,762,762,0,2,2,0,6,6,0,20,20,0,6,6,0,6,6,0
MASS,"100 Bigelow Street,Holyoke MA 01040","(42.202493000000004, -72.60787662257826)",3,6,192,192,0,1536,1536,0,60600,60600,0,381,381,0,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
SALT,"572 Delong St,Salt Lake City, UT 84104","(40.75707505789612, -111.95346637770317)",3,6,118,192,74,1370,1536,166,60230,60600,370,354,381,27,1,2,1,2,2,0,10,10,0,2,2,0,3,3,0
NCSA,"1725 S Oak St.,Champaign, IL 61820","(40.1035624, -88.2415105)",3,6,168,192,24,1504,1536,32,60500,60600,100,378,381,3,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
UTAH,"875 South West Temple,Salt Lake City, UT 84101","(40.7626391, -111.8939563)",5,10,272,320,48,2496,2560,64,116200,116400,200,633,635,2,2,2,0,4,4,0,16,16,0,4,4,0,5,5,0
WASH,"The Bexley, 1761 Old Meadow Road, McLean, VA 22102, United States of America","(38.91930235, -77.21183383681088)",3,6,62,192,130,1344,1536,192,60110,60600,490,367,381,14,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
DALL,"1950 N Stemmons Fwy,Dallas, TX 75207","(32.8002714, -96.8198113)",3,6,158,192,34,1400,1536,136,60410,60600,190,369,381,12,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
MAX,,"(0, 0)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
MICH,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",3,6,132,192,60,1296,1536,240,59040,60600,1560,358,381,23,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
TACC,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261806879021)",5,10,248,320,72,2208,2560,352,114100,116400,2300,596,635,39,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0


## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If all interfaces in the list are located on the same site, the network will automatically be a local Ethernet.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on the same site you can specify the name of the site in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a random site name that can be used for both nodes.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [3]:
slice_name = 'CNERT 1'

site1 = 'GPN'  
site2 = 'SALT'

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
network1_2_name='net1_2'
network2_3_name='net2_3'
node1_nic_name = 'nic1'
node2_1_nic_name = 'nic2_1'
node2_3_nic_name = 'nic2_3'
node3_nic_name = 'nic3'
image='default_ubuntu_20'

cores = 2
ram = 8
disk = 10

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site1, cores=cores, ram=ram, disk=disk)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    node1.set_image(image)
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site1, cores=cores, ram=ram, disk=disk)
    iface2_1 = node2.add_component(model='NIC_Basic', name=node2_1_nic_name).get_interfaces()[0]
    iface2_3 = node2.add_component(model='NIC_Basic', name=node2_3_nic_name).get_interfaces()[0]
    node2.set_image(image)

    # Node3
    node3 = slice.add_node(name=node3_name, site=site2, cores=cores, ram=ram, disk=disk)
    iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]
    node3.set_image(image)
    
    # Network
    net1_2 = slice.add_l2network(name=network1_2_name, interfaces=[iface1, iface2_1])
    net2_3 = slice.add_l2network(name=network2_3_name, interfaces=[iface2_3, iface3])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 7, Time: 215 sec


ID,70324287-8a3f-418f-a4cc-be79f0a5c468
Name,CNERT 1
Lease Expiration (UTC),2023-01-15 19:18:24 +0000
Lease Start (UTC),2023-01-14 19:18:25 +0000
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
0486d965-8ead-4df9-9bcb-e0e38f22d88e,Node1,2,8,10,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe57:5c4d,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe57:5c4d,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
bc2fc4ed-ab2d-45f8-8000-f58c0fea2450,Node2,2,8,10,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe61:e69d,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe61:e69d,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
7b5b9423-a8bf-4157-9533-69089793c39f,Node3,2,8,10,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:feea:8deb,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:feea:8deb,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
1fe46eb6-d67a-4635-9109-ae4f38963464,net1_2,L2,L2Bridge,GPN,None,None,Active,
af4bd670-4a16-4dda-a6c8-67ea39792419,net2_3,L2,L2STS,None,None,None,Active,



Time to stable 215 seconds
Running post_boot_config ...  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
 Time to post boot config 230 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
Node1-nic1-p1,Node1,net1_2,100,None,02:14:79:AE:DA:CC,ens7,ens7
Node2-nic2_3-p1,Node2,net2_3,100,None,0A:21:5F:DC:EC:ED,ens8,ens8
Node2-nic2_1-p1,Node2,net1_2,100,None,06:F8:FE:1C:4E:A5,ens7,ens7
Node3-nic3-p1,Node3,net2_3,100,None,0A:9B:EC:EE:D4:9E,ens7,ens7



Time to print interfaces 240 seconds


## Observe the Slice's Attributes

### Print the slice 

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   CNERT 1
Slice ID     70324287-8a3f-418f-a4cc-be79f0a5c468
Slice State  StableOK
Lease End    2023-01-15 19:18:24 +0000
-----------  ------------------------------------


## Print the Node List

In [6]:
try:
    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
0486d965-8ead-4df9-9bcb-e0e38f22d88e,Node1,2,8,10,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe57:5c4d,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe57:5c4d,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
bc2fc4ed-ab2d-45f8-8000-f58c0fea2450,Node2,2,8,10,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe61:e69d,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe61:e69d,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
7b5b9423-a8bf-4157-9533-69089793c39f,Node3,2,8,10,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:feea:8deb,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:feea:8deb,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key


## Print the Node Details

In [7]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------------
ID                 0486d965-8ead-4df9-9bcb-e0e38f22d88e
Name               Node1
Cores              2
RAM                8
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               gpn-w4.fabric-testbed.net
Site               GPN
Management IP      2610:e0:a04c:fab2:f816:3eff:fe57:5c4d
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe57:5c4d
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------

## Print the Interfaces

In [8]:
try:    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
Node1-nic1-p1,Node1,net1_2,100,None,02:14:79:AE:DA:CC,ens7,ens7
Node2-nic2_3-p1,Node2,net2_3,100,None,0A:21:5F:DC:EC:ED,ens8,ens8
Node2-nic2_1-p1,Node2,net1_2,100,None,06:F8:FE:1C:4E:A5,ens7,ens7
Node3-nic3-p1,Node3,net2_3,100,None,0A:9B:EC:EE:D4:9E,ens7,ens7


## Prolong the lease

In [9]:
import datetime
end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=36)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice.renew(end_date+"+0000")
    print(datetime.datetime.utcnow(),end_date+"+0000")
except Exception as e:
    print(f"Exception: {e}")

2023-01-14 19:23:10.283248 2023-02-19 19:23:07 +0000


##  Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on their new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Pick a Subnet

Create a subnet and list of available IP addresses. All objects are Python IP management objects. You can use either IPv4 or IPv6 subnets and addresses.

In [10]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet1 = IPv4Network("192.168.1.0/24")
    available_ips1 = list(subnet1)[1:]
    subnet2 = IPv4Network("192.168.2.0/24")
    available_ips2 = list(subnet2)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [11]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_2_name) 
    
    stdout, stderr = node1.execute('sudo apt install net-tools')
    print(stdout, stderr)
    stdout, stderr = node1.execute(f'sudo ifconfig {node1_iface.get_os_interface()} up')
    print(stdout, stderr)

    node1_addr = available_ips1.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet1)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

Reading package lists... 

 
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
 Fetched 196 kB in 1s (243 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 63821 files and directories currently installed.)
Prepar

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [12]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_1_iface = node2.get_interface(network_name=network1_2_name)  
    node2_1_addr = available_ips1.pop(0)
    node2_1_iface.ip_addr_add(addr=node2_1_addr, subnet=subnet1)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_1_iface.get_os_interface()}')
    print (stdout)

    stdout, stderr = node2.execute('sudo apt install net-tools')
    print(stdout, stderr)
    stdout, stderr = node2.execute(f'sudo ifconfig {node2_1_iface.get_os_interface()} up')
    print(stdout, stderr)
    
    node2_3_iface = node2.get_interface(network_name=network2_3_name)  
    node2_3_addr = available_ips2.pop(0)
    node2_3_iface.ip_addr_add(addr=node2_3_addr, subnet=subnet2)
    
    stdout, stderr = node2.execute(f'sudo ifconfig {node2_3_iface.get_os_interface()} up')
    print(stdout, stderr)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_3_iface.get_os_interface()}')
    print (stdout)

except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 06:f8:fe:1c:4e:a5 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
3: ens7: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 06:f8:fe:1c:4e:a5 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever

Reading package lists... 

 
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb termina

### Configure Node3

Repeat the steps to add the next available IP to the third node.

In [13]:
try:
    node3 = slice.get_node(name=node3_name)        
    node3_iface = node3.get_interface(network_name=network2_3_name) 
    node3_addr = available_ips2.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=subnet2)

    stdout, stderr = node3.execute('sudo apt install net-tools')
    print(stdout, stderr)
    stdout, stderr = node3.execute(f'sudo ifconfig {node3_iface.get_os_interface()} up')
    print(stdout, stderr)
    
    
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

Reading package lists...
Building dependency tree... 

 
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
 Fetched 196 kB in 1s (350 kB/s)
 dpkg-preconfigure: unable to re-open stdin: 
 Selecting previously unselected package net-tools.
(Reading database ... 63577 files and directories currently installed.)
Prep

In [14]:
node1 = slice.get_node(name=node1_name)
node2 = slice.get_node(name=node2_name) 
node3 = slice.get_node(name=node3_name)
node1_addr = "192.168.1.1"
node2_1_addr = "192.168.1.2"
node2_3_addr = "192.168.2.1"
node3_addr = "192.168.2.2"

## Run the Experiment

We will find the ping round trip time for this pair of sites.


In [15]:
try:
#     node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 15 {node2_1_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=64 time=0.388 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=64 time=0.087 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=64 time=0.090 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=64 time=0.088 ms
64 bytes from 192.168.1.2: icmp_seq=5 ttl=64 time=0.097 ms
64 bytes from 192.168.1.2: icmp_seq=6 ttl=64 time=0.082 ms
64 bytes from 192.168.1.2: icmp_seq=7 ttl=64 time=0.086 ms
64 bytes from 192.168.1.2: icmp_seq=8 ttl=64 time=0.084 ms
64 bytes from 192.168.1.2: icmp_seq=9 ttl=64 time=0.063 ms
64 bytes from 192.168.1.2: icmp_seq=10 ttl=64 time=0.060 ms
64 bytes from 192.168.1.2: icmp_seq=11 ttl=64 time=0.068 ms
64 bytes from 192.168.1.2: icmp_seq=12 ttl=64 time=0.069 ms
64 bytes from 192.168.1.2: icmp_seq=13 ttl=64 time=0.061 ms
64 bytes from 192.168.1.2: icmp_seq=14 ttl=64 time=0.063 ms
64 bytes from 192.168.1.2: icmp_seq=15 ttl=64 time=0.055 ms

--- 192.168.1.2 ping statistics ---
15 packets transmit

In [16]:
try:
#     node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node3.execute(f'ping -c 5 {node2_3_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.2.1 (192.168.2.1) 56(84) bytes of data.
64 bytes from 192.168.2.1: icmp_seq=1 ttl=64 time=46.5 ms
64 bytes from 192.168.2.1: icmp_seq=2 ttl=64 time=23.1 ms
64 bytes from 192.168.2.1: icmp_seq=3 ttl=64 time=23.2 ms
64 bytes from 192.168.2.1: icmp_seq=4 ttl=64 time=23.1 ms
64 bytes from 192.168.2.1: icmp_seq=5 ttl=64 time=23.1 ms

--- 192.168.2.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 23.096/27.786/46.492/9.352 ms
PING 192.168.2.1 (192.168.2.1) 56(84) bytes of data.
64 bytes from 192.168.2.1: icmp_seq=1 ttl=64 time=46.5 ms
64 bytes from 192.168.2.1: icmp_seq=2 ttl=64 time=23.1 ms
64 bytes from 192.168.2.1: icmp_seq=3 ttl=64 time=23.2 ms
64 bytes from 192.168.2.1: icmp_seq=4 ttl=64 time=23.1 ms
64 bytes from 192.168.2.1: icmp_seq=5 ttl=64 time=23.1 ms

--- 192.168.2.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 23.096/27.786/46.492/9.352 ms




In [17]:
try:
#     node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node2.execute(f'ping -c 5 {node3_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.2.2 (192.168.2.2) 56(84) bytes of data.
64 bytes from 192.168.2.2: icmp_seq=1 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=2 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=3 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=4 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=5 ttl=64 time=23.1 ms

--- 192.168.2.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 23.079/23.089/23.108/0.010 ms
PING 192.168.2.2 (192.168.2.2) 56(84) bytes of data.
64 bytes from 192.168.2.2: icmp_seq=1 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=2 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=3 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=4 ttl=64 time=23.1 ms
64 bytes from 192.168.2.2: icmp_seq=5 ttl=64 time=23.1 ms

--- 192.168.2.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 23.079/23.089/23.108/0.010 ms




---

## Installing iperf

In [18]:
config_threads = {}

config_threads[node1] = node1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
config_threads[node2] = node2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
config_threads[node3] = node3.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')

for node, thread in config_threads.items():
    try:
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(stdout)
    except:
        pass

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
 

 Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1948 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [318 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.7 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1449 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [205 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/rest

AttributeError: 'tuple' object has no attribute 'result'

In [ ]:
# stdout, stderr = node1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
# print (stdout)

# stdout, stderr = node2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
# print (stdout)

# stdout, stderr = node3.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
# print (stdout)

## Setting Parameters

In [19]:
stdout, stderr = node1.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq
net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [20]:
stdout, stderr = node2.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq
net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [21]:
stdout, stderr = node3.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq
net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [22]:
#server
stdout, stderr = node2.execute('tmux new -d \'iperf -s -f K\'') 
print (stdout)
print (stderr)

In [23]:
#client
stdout, stderr = node1.execute(f'iperf -c {node2_1_addr} -P 50 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.1.2, TCP port 5001
TCP window size: 1.95 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 12] local 192.168.1.1 port 47222 connected with 192.168.1.2 port 5001
[  3] local 192.168.1.1 port 47204 connected with 192.168.1.2 port 5001
[ 19] local 192.168.1.1 port 47232 connected with 192.168.1.2 port 5001
[ 14] local 192.168.1.1 port 47226 connected with 192.168.1.2 port 5001
[ 23] local 192.168.1.1 port 47242 connected with 192.168.1.2 port 5001
[  4] local 192.168.1.1 port 47206 connected with 192.168.1.2 port 5001
[ 16] local 192.168.1.1 port 47234 connected with 192.168.1.2 port 5001
[ 34] local 192.168.1.1 port 47266 connected with 192.168.1.2 port 5001
[ 11] local 192.168.1.1 port 47220 connected with 192.168.1.2 port 5001
[ 10] local 192.168.1.1 port 47218 connected with 192.168.1.2 port 5001
[ 26] local 192.168.1.1 port 47252 connected with 

In [24]:
#client
stdout, stderr = node3.execute(f'iperf -c {node2_3_addr} -P 50 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.2.1, TCP port 5001
TCP window size: 1.95 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 29] local 192.168.2.2 port 60172 connected with 192.168.2.1 port 5001
[ 10] local 192.168.2.2 port 60132 connected with 192.168.2.1 port 5001
[ 30] local 192.168.2.2 port 60174 connected with 192.168.2.1 port 5001
[  9] local 192.168.2.2 port 60134 connected with 192.168.2.1 port 5001
[ 14] local 192.168.2.2 port 60142 connected with 192.168.2.1 port 5001
[  3] local 192.168.2.2 port 60120 connected with 192.168.2.1 port 5001
[ 49] local 192.168.2.2 port 60212 connected with 192.168.2.1 port 5001
[ 24] local 192.168.2.2 port 60162 connected with 192.168.2.1 port 5001
[ 19] local 192.168.2.2 port 60152 connected with 192.168.2.1 port 5001
[ 28] local 192.168.2.2 port 60170 connected with 192.168.2.1 port 5001
[ 26] local 192.168.2.2 port 60166 connected with 

In [25]:
#client
stdout, stderr = node3.execute(f'iperf -c {node2_3_addr} -P 50 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.2.1, TCP port 5001
TCP window size: 1.95 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 27] local 192.168.2.2 port 60272 connected with 192.168.2.1 port 5001
[  5] local 192.168.2.2 port 60228 connected with 192.168.2.1 port 5001
[  4] local 192.168.2.2 port 60226 connected with 192.168.2.1 port 5001
[  3] local 192.168.2.2 port 60224 connected with 192.168.2.1 port 5001
[ 10] local 192.168.2.2 port 60238 connected with 192.168.2.1 port 5001
[  7] local 192.168.2.2 port 60232 connected with 192.168.2.1 port 5001
[ 52] local 192.168.2.2 port 60322 connected with 192.168.2.1 port 5001
[  6] local 192.168.2.2 port 60230 connected with 192.168.2.1 port 5001
[ 11] local 192.168.2.2 port 60240 connected with 192.168.2.1 port 5001
[  9] local 192.168.2.2 port 60236 connected with 192.168.2.1 port 5001
[ 46] local 192.168.2.2 port 60310 connected with 

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")